In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
import h5py
from collections import Counter
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

stemmer = SnowballStemmer("english", ignore_stopwords=True)
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')
np.set_printoptions(threshold=np.nan)

/Users/eunbeejang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load data
IMDB_train = pd.read_csv('./IMDB-train.txt', sep='\t', encoding='latin-1', header=None)
IMDB_train_y = IMDB_train[:][1]
IMDB_valid = pd.read_csv('./IMDB-valid.txt', sep='\t', encoding='latin-1', header=None)
IMDB_valid_y = IMDB_valid[:][1]
IMDB_test = pd.read_csv('./IMDB-test.txt', sep='\t', encoding='latin-1', header=None)
IMDB_test_y = IMDB_test[:][1]
stemmer = SnowballStemmer("english", ignore_stopwords=True)

print("Data loaded.")

Data loaded.


In [3]:
frames = [IMDB_train, IMDB_valid]
frames_y = [IMDB_train_y, IMDB_valid_y]
IMDB_train = pd.concat(frames)
IMDB_train_y = pd.concat(frames_y)

In [141]:
#IMDB_train = IMDB_train[:][0]
#IMDB_test = IMDB_test[:][0]

In [142]:
IMDB_train[0]

0       For a movie that gets no respect there sure ar...
1       Bizarre horror movie filled with famous faces ...
2       A solid, if unremarkable film. Matthau, as Ein...
3       It's a strange feeling to sit alone in a theat...
4       You probably all already know this by now, but...
5       I saw the movie with two grown children. Altho...
6       You're using the IMDb.<br /><br />You've given...
7       This was a good film with a powerful message o...
8       Made after QUARTET was, TRIO continued the qua...
9       For a mature man, to admit that he shed a tear...
10      Aileen Gonsalves, my girlfriend, is in this fi...
11      Jonathan Demme's directorial debut for Roger C...
12      When I rented this movie to watch it, I knew t...
13      It's hard to say sometimes why exactly a film ...
14      Yes, this gets the full ten stars. It's plain ...
15      Hello. This movie is.......well.......okay. Ju...
16      This is a film that was very well done. I had ...
17      A typi

# Preprocessing

In [4]:
def preprocessing(data):
    new_data = []
    #i = 0
    for sentence in (data[:][0]):
        #clean = re.compile('<.*?>')
        new_sentence = re.sub('<.*?>', '', sentence) # remove HTML tags
        new_sentence = re.sub(r'[^\w\s]', '', new_sentence) # remove punctuation
        new_sentence = new_sentence.lower() # convert to lower case
        if new_sentence != '':
            new_data.append(new_sentence)
    return new_data

In [144]:
IMDB_train

,0,1
0,For a movie that gets no respect there sure ar...,1
1,Bizarre horror movie filled with famous faces ...,1
2,"A solid, if unremarkable film. Matthau, as Ein...",1
3,It's a strange feeling to sit alone in a theat...,1
4,"You probably all already know this by now, but...",1
5,I saw the movie with two grown children. Altho...,1
6,You're using the IMDb.<br /><br />You've given...,1
7,This was a good film with a powerful message o...,1
8,"Made after QUARTET was, TRIO continued the qua...",1
9,"For a mature man, to admit that he shed a tear...",1


In [145]:
"""
def rm_stopwords(data):
    new_data = []
    for sent in data:
        new_data.append([w for w in sent if w not in stopwords])
    return new_data
"""

'\ndef rm_stopwords(data):\n    new_data = []\n    for sent in data:\n        new_data.append([w for w in sent if w not in stopwords])\n    return new_data\n'

In [146]:
"""
<<<<IGNORE THESE FOR NOW>>>>

def tokenize(data):
    new_data = []
    for sentence in (data):
        new_sentence = nltk.word_tokenize(sentence)
        new_data.append(new_sentence)
    return new_data        

def stem_lem(data):
    new_data = []
    for sent in data:
        this_sent = []
        for w in test3:
            w = stemmer.stem(w)
            w = lemmatizer.lemmatize(w)
            this_sent.append(stemmer.stem(w))
        new_data.append(this_sent)
    return new_data
"""

'\n<<<<IGNORE THESE FOR NOW>>>>\n\ndef tokenize(data):\n    new_data = []\n    for sentence in (data):\n        new_sentence = nltk.word_tokenize(sentence)\n        new_data.append(new_sentence)\n    return new_data        \n\ndef stem_lem(data):\n    new_data = []\n    for sent in data:\n        this_sent = []\n        for w in test3:\n            w = stemmer.stem(w)\n            w = lemmatizer.lemmatize(w)\n            this_sent.append(stemmer.stem(w))\n        new_data.append(this_sent)\n    return new_data\n'

In [5]:
IMDB_train = preprocessing(IMDB_train)
#IMDB_valid = preprocessing(IMDB_valid)
IMDB_test = preprocessing(IMDB_test)

In [148]:
#IMDB_train_tok = tokenize(IMDB_train)
#IMDB_valid_tok = tokenize(IMDB_valid)
#IMDB_test_tok = tokenize(IMDB_test)

In [133]:
#IMDB_train_sl = stem_lem(IMDB_train_tok)
#IMDB_valid_sl = stem_lem(IMDB_valid_tok)
#IMDB_test_sl = stem_lem(IMDB_test_tok)

In [134]:
#IMDB_train_stop = rm_stopwords(IMDB_train_sl)
#IMDB_valid_stop = rm_stopwords(IMDB_valid_sl)
#IMDB_test_stop = rm_stopwords(IMDB_test_sl)

# Bag of n-gram 

In [6]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [7]:

unigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words='english', max_features =30000)
bigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(2, 2), stop_words='english', max_features =30000)
trigram = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(3, 3), stop_words='english', max_features =30000)



In [8]:
train_unigram = unigram.fit_transform(IMDB_train).toarray()


In [14]:
def rm_sents(data, target):
    new_data = []
    new_target = []
    for i in range(0,len(data)):
        if len(list(set(data[i]))) is not 1:
            new_data.append(data[i])
            new_target.append(target[i])
    return new_data, new_target

In [21]:
new_data, n_y = rm_sents(train_unigram, np.asarray(IMDB_train_y))

1

In [102]:
train_unigram = unigram.fit_transform(IMDB_train).toarray()
test_unigram = unigram.transform(IMDB_test).toarray()


In [137]:
train_unigram.shape

(25000, 30000)

In [75]:
train_bigram = bigram.fit_transform(IMDB_train).toarray()
test_bigram = bigram.transform(IMDB_test).toarray()

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [22]:
train_trigram = trigram.fit_transform(IMDB_train).toarray()
test_trigram = trigram.transform(IMDB_test).toarray()

In [26]:
type(test_bigram)
np.save('./train_unigram', train_unigram)

In [29]:
with h5py.File('train_unigram.hdf5', 'w') as f:
    dset = f.create_dataset("default", data=train_unigram)

In [34]:
#pd.DataFrame(train_unigram).to_csv('./train_unigram.csv', index = False, header = False)
#pd.DataFrame.from_records(train_bigram).to_csv('./train_bigram.csv', index = False, header = False)
#pd.DataFrame.from_records(train_trigram).to_csv('./train_trigram.csv', index = False, header = False)


In [30]:

unigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(1, 1), stop_words=None, max_features =30000)
bigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(2, 2), stop_words=None, max_features =30000)
trigram_w_sw = CountVectorizer(tokenizer=LemmaTokenizer(), analyzer='word', ngram_range=(3, 3), stop_words=None, max_features =30000)



In [31]:
train_unigram_w_sw = unigram_w_sw.fit_transform(IMDB_train).toarray()
test_unigram_w_sw = unigram_w_sw.transform(IMDB_test).toarray()


In [32]:
train_bigram_w_sw = bigram_w_sw.fit_transform(IMDB_train).toarray()
test_bigram_w_sw = bigram_w_sw.transform(IMDB_test).toarray()


In [33]:
train_trigram_w_sw = trigram_w_sw.fit_transform(IMDB_train).toarray()
test_trigram_w_sw = trigram_w_sw.transform(IMDB_test).toarray()

In [ ]:
pd.DataFrame.from_records(train_unigram_w_sw).to_csv('./train_unigram_w_sw.csv', index = False, header = False)
pd.DataFrame.from_records(train_bigram_w_sw).to_csv('./train_bigram_w_sw.csv', index = False, header = False)
pd.DataFrame.from_records(train_trigram_w_sw).to_csv('./train_trigram_w_sw.csv', index = False, header = False)
